In [1]:
import torch
import torch.nn as nn
import os
import numpy as np
import cv2
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Dataset

In [3]:
animals = {0: 'butterfly', 1: 'cat', 2: 'cow', 3: 'dog', 4: 'elephant', 5: 'hen', 6: 'horse', 7: 'sheep', 8: 'spider', 9: 'squirrel'}
rev_animals = {j:i for i,j in animals.items()}
print(rev_animals)

{'butterfly': 0, 'cat': 1, 'cow': 2, 'dog': 3, 'elephant': 4, 'hen': 5, 'horse': 6, 'sheep': 7, 'spider': 8, 'squirrel': 9}


In [4]:
imgx, imgy = 100, 100
bs = 20000

In [5]:
path_to_main_dir = '/home/arjun/Desktop/Datasets/animals'
X,y = [], []
for sub in os.listdir(path_to_main_dir):
    for img in os.listdir(os.path.join(path_to_main_dir,sub))[:100]:
        image = cv2.imread(os.path.join(os.path.join(path_to_main_dir,sub), img))
        x = cv2.resize(image, (imgx, imgy))
        X.append(x)
        y.append(rev_animals[sub])
len(X)

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known inc

In [7]:
X = torch.tensor(X).to(device).float()
y = np.array(y)
y = F.one_hot(torch.tensor(y), num_classes=10).to(device).float() # Super slow :(

/tmp/ipykernel_74752/2460507321.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  X = torch.tensor(X).to(device).float()


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=.8)

### Converting to DataLoader

In [9]:
class Dataset:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [10]:
train = Dataset(X_train, y_train)
test = Dataset(X_test, y_test)
len(train), len(test)

(14929, 3733)

In [11]:
train_dataloader = DataLoader(train, bs, True)
test_dataloader = DataLoader(test, bs, True)

# Creating Model

In [13]:
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)


    def forward(self, X):
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 5, 5
        x = x.view(-1, 16 * 5 * 5)            # -> n, 400
        x = F.relu(self.fc1(x))               # -> n, 120
        x = F.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                       # -> n, 10
        return x 


model = CNN().to(device)
model

MLP(
  (lay1): Linear(in_features=30000, out_features=5000, bias=True)
  (lay2): ReLU()
  (lay3): Linear(in_features=5000, out_features=1000, bias=True)
  (lay4): ReLU()
  (lay5): Linear(in_features=1000, out_features=100, bias=True)
  (lay6): ReLU()
  (lay7): Linear(in_features=100, out_features=10, bias=True)
  (drop1): DropoutLayer()
  (drop2): DropoutLayer()
)

In [18]:
lossCat = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr=1e-5)

# Training

In [22]:

for epoch in range(100):
    for step, (x,y) in enumerate(train_dataloader):
        # forward pass
        x = x.reshape(-1, imgx*imgy*3)
        y_pred = model(x)
        
        # Loss calculation
        loss = lossCat(y_pred, y)
        
        # Backpropogation
        loss.backward()
        optimiser.step()
        optimiser.zero_grad()
    
    if epoch%10 == 0:    
        print('Epoch:', epoch, ':', loss.item())
    if loss.item() < 1: break

Epoch: 0 : 1.7294049263000488
Epoch: 10 : 1.7443870306015015
Epoch: 20 : 1.6012189388275146
Epoch: 30 : 1.5015195608139038
Epoch: 40 : 1.444165587425232
Epoch: 50 : 1.3414175510406494
Epoch: 60 : 1.3261395692825317
Epoch: 70 : 1.4208805561065674
Epoch: 80 : 1.2928086519241333
Epoch: 90 : 1.1975133419036865


# Model Evaluation

### Train Data

In [20]:
tot, correct = 0, 0
for x,y in train_dataloader:
    x = x.reshape(-1, imgx*imgy*3)
    y_pred = model(x)
    correct += sum(torch.argmax(y_pred,dim=1) == torch.argmax(y,dim=1))
    tot += len(y)

print("Accuracy:", (correct/tot*100).item(),'%')
tot
    

Accuracy: 40.17683792114258 %


14929

### Test Data

In [21]:
tot, correct = 0, 0
for x,y in test_dataloader:
    x = x.reshape(-1, imgx*imgy*3)
    y_pred = model(x)
    correct += sum(torch.argmax(y_pred,dim=1) == torch.argmax(y,dim=1))
    tot += len(y)

print("Accuracy:", (correct/tot*100).item(),'%')
tot

Accuracy: 31.15456771850586 %


3733

Best Accuracy: 39% :(